# Experiment 2: Adoption Scenarios

### Description

Simulate different adoption scenarios for clients and hosts, using parameter sweeps.

### Assumptions


### To do:
* Create custom scenarios (in the form of an experiment template) that can easily be subbed in and out.

In [44]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pprint import pprint

import visualizations
from experiments.run import run
from experiments.utils import display_code

time: 23.9 ms (started: 2021-09-16 17:16:01 +02:00)


In [45]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = True

time: 21 ms (started: 2021-09-16 17:16:01 +02:00)


In [46]:
# Import experiment templates
import experiments.templates.time_domain_analysis as time_domain_analysis

time: 20.8 ms (started: 2021-09-16 17:16:01 +02:00)


In [47]:
# Inspect experiment template
#display_code(time_domain_analysis)

time: 26.1 ms (started: 2021-09-16 17:16:01 +02:00)


In [48]:
# Create a new copy of the relevant simulation for each analysis
simulation_1 = copy.deepcopy(time_domain_analysis.experiment.simulations[0])

time: 27.1 ms (started: 2021-09-16 17:16:01 +02:00)


# Adjust experiment

### Override Initial State

In [49]:
# Check initial state:
pprint(simulation_1.model.initial_state)

{'avg_price': 2,
 'clients': 10,
 'hosts': 1,
 'hosts_daily_profit': 0,
 'hosts_daily_revenue': 0,
 'indicated_network_demand': 0,
 'network_allocation': 0,
 'network_capacity': 1,
 'network_penetration': 0,
 'platform_daily_revenue': 0,
 'potential_users': 10000}
time: 25.8 ms (started: 2021-09-16 17:16:01 +02:00)


In [50]:
simulation_1.model.initial_state.update({
    "hosts": 1, 
    "clients": 10
})

time: 39.5 ms (started: 2021-09-16 17:16:01 +02:00)


### Override parameters

In [51]:
pprint(simulation_1.model.params)

{'MIN_expected_fulfillment': [0.33],
 'avg_client_allocation': [10],
 'avg_host_line': [1000],
 'avg_reserve_capacity': [0.25],
 'client_competitor_price': [2],
 'client_registration_delay': [14],
 'host_line_cost': [0.05],
 'host_setup_delay': [60],
 'host_technical_difficulty': [0.5],
 'initial_population': [10000],
 'network_inefficiencies': [0.2],
 'onboarding_coefficient': [0.75],
 'price_change_delay': [14],
 'service_fee': [0.05]}
time: 23.5 ms (started: 2021-09-16 17:16:01 +02:00)


In [52]:
simulation_1.model.params.update({
    "host_setup_delay": [14, 30, 90],
    "client_registration_delay": [21],
    #"onboarding_coefficient": [0.5, 0.75, 1]
})

time: 26.3 ms (started: 2021-09-16 17:16:01 +02:00)


In [53]:
# Execute the simulation (radCAD):
df_1, _exceptions = run(simulation_1)

time: 185 ms (started: 2021-09-16 17:16:01 +02:00)


In [54]:
# View DataFrame:
#df_1.head(100)

time: 23.6 ms (started: 2021-09-16 17:16:01 +02:00)


# Anaysis 1: User Adoption

Plot user adoption over the time horizon.

The rate of user adoption at any given timestep is a function of: 
- **word-of-mouth** (the more clients there are, the more likely )
- **base onboarding rate** (set in system parameters as 'onboarding_coeff')
- **price attractiveness** for the given agent (i.e. lower price = more attractive to clients but less attractive for hosts) 
- **registration delay** (i.e the average time clients take to register)

In [55]:
# fig_df = df_1.query('variable_name == 2')

fig = px.line(
    df_1,
    x='timestep',
    y=['hosts', 'clients'],
    height=400,
    animation_frame='host_setup_delay'
)

fig.update_layout(
    title="User Adoption",
    xaxis_title="Days",
    yaxis_title="Persons",
    legend_title="User Type")

fig.show()

time: 102 ms (started: 2021-09-16 17:16:01 +02:00)


# Anaysis 2: Network Allocation

Plot the actual bandwidth allocation for the network over time.

Notes: 
- Since the network assumes clients can initiate temporary connections, actual network allocation is dependent on the current price attractiveness to existing clients, v.s. the 

In [56]:
fig = px.line(
    df_1,
    x='timestep',
    y=['network_allocation'],
    height=350,
    animation_frame='host_setup_delay'
)

fig.update_layout(
    title="Network Allocation",
    xaxis_title="Days",
    yaxis_title="Mbps",
    legend_title="Allocation")

fig.show()

time: 122 ms (started: 2021-09-16 17:16:01 +02:00)


In [57]:
# Use Pandas to query results 

#E.g. maximum network allocation
df_1.groupby('subset')['network_allocation'].max()

subset
0    1.623156e+06
1    1.152232e+06
2    5.818301e+05
Name: network_allocation, dtype: float64

time: 24.1 ms (started: 2021-09-16 17:16:02 +02:00)


In [63]:
fig = px.line(
    df_1,
    x='timestep',
    y=['avg_price'],
    height=300,
    animation_frame='host_setup_delay'
)

fig.update_layout(
    title="AVG Host Price over Time",
    xaxis_title="Days",
    yaxis_title="ZAR/Mbps/Day",
    legend_title="Price")

fig.show()

time: 91.7 ms (started: 2021-09-16 17:16:52 +02:00)


In [59]:
# Plot 3D Surface:

# visualizations.plot_surface(df_1)

time: 26.1 ms (started: 2021-09-16 17:16:02 +02:00)


In [65]:
fig = px.line(
    df_1,
    x='timestep',
    y=['network_penetration'],
    height=300,
    animation_frame='host_setup_delay'
)

fig.update_layout(
    title="Network Penetration",
    xaxis_title="Days",
    yaxis_title="%",
    legend_title="%")

fig.show()

time: 84.6 ms (started: 2021-09-16 17:17:03 +02:00)


# Anaysis: Yields

Host and platform yeilds

In [66]:
fig = px.line(
    df_1,
    x='timestep',
    y=['hosts_daily_revenue', 'hosts_daily_profit', 'platform_daily_revenue'],
    height=400,
    animation_frame='host_setup_delay'
)

fig.update_layout(
    title="Daily Revenue & Profit Yields",
    xaxis_title="Days",
    yaxis_title="ZAR/Day",
    legend_title="Agents"
)

newnames = {'hosts_daily_revenue':'Hosts (revenue)', 'hosts_daily_profit': 'Hosts (profit)', 'platform_daily_revenue': 'Platform (revenue)'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
        
fig.show()

time: 130 ms (started: 2021-09-16 17:17:10 +02:00)


# Analysis: Hosts & Platform Yields

In [67]:
# Simulation metrics
period = df_1['timestep'].max()
service_fee = simulation_1.model.params["service_fee"][0]

# Pandas calculations:
host_cumulative_revenue = int(df_1['hosts_daily_revenue'].sum())
host_cumulative_profit = int(df_1['hosts_daily_profit'].sum())
platform_cumulative_revenue = int(df_1['platform_daily_revenue'].sum())

# Output formatting:
service_fee_formatted = service_fee * 100
revenue_formatted = "R{:,.0f}".format(host_cumulative_revenue)
profit_formatted = "R{:,.0f}".format(host_cumulative_profit)
platform_revenue_formatted = "R{:,.0f}".format(platform_cumulative_revenue)

# Display output:
print("Host cumulative revenue (", period , "days ):")
print(revenue_formatted, "\n")

print("Host cumulative profit (", period , "days ):")
print(profit_formatted, "\n")

print("Platform cumulative revenue (", service_fee_formatted, "% fee) (", period , "days ):")
print(platform_revenue_formatted, "\n")

Host cumulative revenue ( 365 days ):
R156,146,034 

Host cumulative profit ( 365 days ):
R58,426,824 

Platform cumulative revenue ( 5.0 % fee) ( 365 days ):
R7,758,631 

time: 24.1 ms (started: 2021-09-16 17:17:39 +02:00)
